# Машинное обучение и майнинг данных
## 02/02/2017 Введение в sklearn, kNN

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16,8)

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

<img src='https://s-media-cache-ak0.pinimg.com/originals/ac/29/f4/ac29f41d112cb20512ca6d7ce6b00fa6.jpg'>

[Scikit-learn](http://scikit-learn.org/stable/index.html) (или sklearn) - самая популярная библиотека с алгоритмами машинного обучения и практически всеми необходимыми вспомогательными функциями, которые могут понадобится при подготовке данных, визуализации и оценке качества методов.

На странице модуля можно найти примеры для многих методов с пояснениями.

Рвемся в бой

## Стоимость жилья в Калифорнии

In [ ]:
from sklearn.datasets import fetch_california_housing
import mpl_toolkits.basemap as bm

In [ ]:
data = fetch_california_housing()

X = data.data
y = data.target

In [ ]:
print data.DESCR

In [ ]:
# Возьмем подвыборку
lllat, lllon = 33, -125
urlat, urlon = 42, -114

idx = (X[:, -1] <= urlon) & (X[:, -1] >= lllon) &\
      (X[:, -2] <= urlat) & (X[:, -2] >= lllat)

X = X[idx]
y = y[idx]

In [ ]:
# Нарисуем это дело

plt.figure(figsize=(10, 10))

m = bm.Basemap(
    llcrnrlon=lllon,
    llcrnrlat=lllat,
    urcrnrlon=urlon,
    urcrnrlat=urlat, 
    projection='merc',
    resolution='h'
)

m.drawcoastlines(linewidth=0.5)
m.drawmapboundary(fill_color='#47A4C9', zorder=1)
m.fillcontinents(color='#EBC4D8',lake_color='#47A4C9', zorder=2)

parallels = np.linspace(lllat, urlat, 10)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.linspace(lllon, urlon, 10)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)

m.scatter(X[:, -1], X[:, -2], latlon=True, cmap=plt.cm.hot,
          zorder=3, lw=0, c=y)

### Обучим  KNN

Модели в sklearn - это экземпляры класса в `python`, основными методами которого являются `.fit()`, `.predict()` и иногда `.predict_proba()`

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

RND_SEED = 123

In [ ]:
# Разбиваем случайным образом на обучение и контроль

X_train, X_test, y_train, y_test = train_test_split(X[:,-2:], y, test_size=0.3, random_state=RND_SEED)

In [ ]:
knn = KNeighborsRegressor()



### Подбор числа соседей

Зафиксируем все гиперпараметры, кроме числа ближайших соседей.

Переберите значение `k` от 1 до 15, выведите на графике ошибку при каждом `k`.

В будущем, естественно, мы изучем подходы, которые позволяют автоматизировано и эффективно определяеть оптимальные гиперпараметры моделей - например с помощью `GridSearch`, `RandomizedSearch` или [hyperopt](https://github.com/hyperopt/hyperopt).

In [ ]:
## Your code here

Какая методологическая ошибка здесь допущена?

### Оценка обобщающей способности и выбор гиперпараметров

#### Train + Test

До что мы сделали выше - называется метод отложенной выборки

<img src='https://www.lucypark.kr/courses/2015-dm/images/partition.png' width=700>

Случайным образом разбиваем исходные данные в некоторой пропорции (70/30, 80/20) на обучающую и контрольную выборку.

На одной алгоритм обучается, на другой проверяется его качество.

#### Train + Validation + Test

Иногда встречается трехступенчатая стратегия - теперь добавляется так называемое валидационной множество.

Такая схема подходит если данных достаточно много. <br/> 
Оценка качества надежнее чем в предыдущем случае

<img src='https://dataengr.files.wordpress.com/2016/01/partitions.jpg?w=656'>

#### Cross-validation

В предыдущих случаях не все объекты будут участвовать в обучении.

Можно разбить данные на $k$ равных частей, одну часть использовать для тестирования, а все остальное - для обучения. <br/>
Далее качество по всем фолдам усредняется

<img src='http://blog-test.goldenhelix.com/wp-content/uploads/2015/04/B-fig-1.jpg'>

К сожалению, данный подход можно применить не всегда.

## Данные с оттоком клиентов

Загрузите [данные](https://raw.githubusercontent.com/EricChiang/churn/master/data/churn.csv) об оттоке клиентов через pandas.

In [ ]:
df_churn = pd.read_csv('https://raw.githubusercontent.com/EricChiang/churn/master/data/churn.csv')

In [ ]:
df_churn.head().T

In [ ]:
print df_churn.shape

Это небольшие данные, но у них есть большие проблемы:

#### Проблема 1: Признаки разных шкал

In [ ]:
df_churn.describe(include='all').T

#### Проблема 2: Региональные признак
Он может влиять на ответ, но развертывать его в бинарный не имеет смысла.

Можно ли его как-то неявно включить в модель?

In [ ]:
df_churn.loc[:, 'State'].nunique()

#### Проблема 3: Довольно сильный перекос классов

In [ ]:
df_churn.loc[:, 'Churn?'].value_counts()

#### Задание

1. Подготовьте данные 
    * Удалите ненужные признаки
    * Преобразуйте категориальные и бинарные признаки
    * Отнормируйте признаки
2. Выберите меру качества и метод оценки обобщающей способности
3. Вновь, зафиксируйте все гиперпараметры, кроме количества ближайших соседей, найдите наилучшее значение

In [ ]:
## Your code here